### Выполнили студенты группы 6403-010302D Степанов Матвей и Овчинников Егор

In [670]:
import numpy as np
from sympy import Symbol, div

In [671]:
n, k = 7, 4

In [672]:
x = Symbol('x')
g = 1 + x**2 + x**3

### Циклический код

#### Функция кодирования циклического кода

In [673]:
def encode(a, g, n):
    a_polynom = 0
    for idx, el in enumerate(a):
        a_polynom += el * x**idx

    coef_dict = dict((a_polynom * g).expand().as_coefficients_dict().items())
    received_word = []
    for i in range(n):
        received_word.append((coef_dict.setdefault(x**i, 0)) % 2)
    return received_word    

#### Кодируем

In [674]:
word = np.array([1, 0, 0, 1])
encoded_word = encode(word, g, n)

#### Функция декодирования циклического кода

In [675]:
def decode(a, g, n, k: int, t: int):
    a_polynom = 0
    for idx, el in enumerate(a):
        a_polynom += el * x**idx

    r = div(a_polynom, g)[1]
    coef_dict = dict(r.as_coefficients_dict().items())
    syndrome = 0
    for i in range(n):
        syndrome += ((coef_dict.setdefault(x**i, 0) % 2) * x**i)


    for i in range(n):
        r = div(x**i * syndrome, g)[1]
        coef_dict = dict(r.as_coefficients_dict().items())
        syndrome_i = 0
        for j in range(n):
            syndrome_i += ((coef_dict.setdefault(x**j, 0) % 2) * x**j)

        if sum(syndrome_i.as_coefficients_dict().values()) <= t:
            e = (x**(n - i) * syndrome_i).expand()
            result_polynom = a_polynom + e

            coef_dict = dict(result_polynom.as_coefficients_dict().items())
            result_polynom = 0
            for l in range(n):
                result_polynom += ((coef_dict.setdefault(x**l, 0)) % 2) * x**l

            coef_dict = dict(div(result_polynom, g)[0].as_coefficients_dict().items())
            result_word = []
            for l in range(k):
                result_word.append((coef_dict.setdefault(x**l, 0)) % 2)

            return np.array(result_word, dtype=int)    
            
    return None


#### Однократная ошибка

In [676]:
broken_word = encoded_word.copy()
broken_word[0] ^= 1

all(decode(broken_word, g, n, k, 1) == word)

True

#### Двукратная ошибка

In [677]:
broken_word = encoded_word.copy()
broken_word[2] ^= 1
broken_word[3] ^= 1

all(decode(broken_word, g, n, k, 2) == word)

False

#### Трёхкратная ошибка

In [678]:
broken_word = encoded_word.copy()
broken_word[2] ^= 1
broken_word[3] ^= 1
broken_word[4] ^= 1

all(decode(broken_word, g, n, k, 3) == word)

False

### Циклический код через пакеты ошибок

In [679]:
g = 1 + x ** 3 + x ** 4 + x ** 5 + x ** 6
n, k = 15, 9

E = np.array([1, 1 + x, 1 + x**2, 1 + x + x**2])

#### Функция декодирования

In [680]:
def decode(a, g, n, k: int, t: int):
    a_polynom = 0
    for idx, el in enumerate(a):
        a_polynom += el * x**idx

    r = div(a_polynom, g)[1]
    coef_dict = dict(r.as_coefficients_dict().items())
    syndrome = 0
    for i in range(n):
        syndrome += ((coef_dict.setdefault(x**i, 0) % 2) * x**i)

    for i in range(n):
        r = div(x**i * syndrome, g)[1]

        coef_dict = dict(r.as_coefficients_dict().items())
        r = 0
        for j in range(n):
            r += ((coef_dict.setdefault(x**j, 0) % 2) * x**j)

        # print(r)
        if r in E:
            syndrome_i = r
            e = (x**(n - i) * syndrome_i).expand()
            result_polynom = a_polynom + e

            coef_dict = dict(result_polynom.as_coefficients_dict().items())
            result_polynom = 0
            for l in range(n):
                result_polynom += ((coef_dict.setdefault(x**l, 0)) % 2) * x**l

            coef_dict = dict(div(result_polynom, g)[0].as_coefficients_dict().items())
            result_word = []
            for l in range(k):
                result_word.append((coef_dict.setdefault(x**l, 0)) % 2)

            return np.array(result_word, dtype=int) 
            
    return None  

#### Однократная ошибка

In [681]:
word = np.array([1, 0, 0, 1, 0, 0, 0, 1, 1])
encoded_word = encode(word, g, n)
broken_word = encoded_word.copy()

broken_word[5] ^= 1

all(decode(broken_word, g, n, k, 1) == word)

True

#### Двукратная ошибка

In [682]:
broken_word = encoded_word.copy()

broken_word[5] ^= 1
broken_word[7] ^= 1

all(decode(broken_word, g, n, k, 2) == word)

True

#### Трёхкратная ошибка

In [683]:
broken_word = encoded_word.copy()

broken_word[5] ^= 1
broken_word[6] ^= 1
broken_word[7] ^= 1

all(decode(broken_word, g, n, k, 3) == word)

True

#### Четырёхкратная ошибка

In [684]:
broken_word = encoded_word.copy()

broken_word[5] ^= 1
broken_word[6] ^= 1
broken_word[7] ^= 1
broken_word[8] ^= 1

all(decode(broken_word, g, n, k, 3) == word)

False